In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 63.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from transformers import BertTokenizer, TFBertForSequenceClassification, BertConfig
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from google.colab import drive
import numpy as np

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = '/content/drive/My Drive/Sentiment Analysis/emoevent_es.csv'
df = pd.read_csv(file_path, delimiter='\t')
label_encoder = LabelEncoder()
df['emotion'] = label_encoder.fit_transform(df['emotion'])

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenized = tokenizer(df['tweet'].tolist(), max_length=128, padding='max_length', truncation=True, return_tensors='tf')

(…)cased/resolve/main/tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

(…)bert-base-uncased/resolve/main/vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

(…)base-uncased/resolve/main/tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

(…)rt-base-uncased/resolve/main/config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
X = tokenized['input_ids'].numpy()
y = df['emotion'].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
config = BertConfig.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', config=config)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
optimizer = Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
211/211 [==============================] - 218s 815ms/step - loss: 1.3627 - accuracy: 0.5078
Epoch 2/10
211/211 [==============================] - 174s 824ms/step - loss: 1.1135 - accuracy: 0.6160
Epoch 3/10
211/211 [==============================] - 174s 825ms/step - loss: 0.9137 - accuracy: 0.6802
Epoch 4/10
211/211 [==============================] - 174s 825ms/step - loss: 0.7085 - accuracy: 0.7559
Epoch 5/10
211/211 [==============================] - 174s 825ms/step - loss: 0.5257 - accuracy: 0.8258
Epoch 6/10
211/211 [==============================] - 174s 825ms/step - loss: 0.3657 - accuracy: 0.8799
Epoch 7/10
211/211 [==============================] - 174s 825ms/step - loss: 0.2586 - accuracy: 0.9179
Epoch 8/10
211/211 [==============================] - 174s 825ms/step - loss: 0.1788 - accuracy: 0.9423
Epoch 9/10
211/211 [==============================] - 174s 824ms/step - loss: 0.1422 - accuracy: 0.9554
Epoch 10/10
211/211 [==============================] - 174s 824m

In [ ]:
predictions = model.predict(X_test).logits
predicted_labels = [np.argmax(p) for p in predictions]
print(classification_report(y_test, predicted_labels))

53/53 [==============================] - 17s 272ms/step
              precision    recall  f1-score   support

           0       0.52      0.46      0.49       180
           1       0.12      0.07      0.09        30
           2       0.55      0.46      0.50        13
           3       0.51      0.59      0.54       332
           4       0.72      0.74      0.73       835
           5       0.74      0.67      0.70       229
           6       0.21      0.13      0.16        63

    accuracy                           0.63      1682
   macro avg       0.48      0.44      0.46      1682
weighted avg       0.63      0.63      0.63      1682



In [ ]:
print(label_encoder.classes_)

['anger' 'disgust' 'fear' 'joy' 'others' 'sadness' 'surprise']
